In [1]:
import polaris as po
import numpy as np
from rdkit import RDLogger,Chem                                                                                                                                                               
RDLogger.DisableLog('rdApp.*')

# Load the competition from the Hub
competition = po.load_competition("asap-discovery/antiviral-ligand-poses-2025")

# Get the train and test data-loaders
train, test = competition.get_train_test_split()

#these are aligned (to provided reference) mols.
#for their generation, see the respective notebooks
sarsmols = [m for m in Chem.SDMolSupplier("sarsmols.sdf")]
mersmols = [m for m in Chem.SDMolSupplier("mersmols.sdf")]

[2025-03-12 18:00:53] INFO     The version of Polaris that was used to create the artifact (0.11.0) ]8;id=457061;file:///home/wim/anaconda3/envs/my-rdkit-env/lib/python3.12/site-packages/polaris/_artifact.py\_artifact.py]8;;\:]8;id=502578;file:///home/wim/anaconda3/envs/my-rdkit-env/lib/python3.12/site-packages/polaris/_artifact.py#92\92]8;;\
                               is different from the currently installed version of Polaris                        
                               (0.11.9).                                                                           

                      WARNING  You're loading data from a remote location. If the dataset is small     ]8;id=926301;file:///home/wim/anaconda3/envs/my-rdkit-env/lib/python3.12/site-packages/polaris/dataset/_base.py\_base.py]8;;\:]8;id=324522;file:///home/wim/anaconda3/envs/my-rdkit-env/lib/python3.12/site-packages/polaris/dataset/_base.py#181\181]8;;\
                               enough, consider caching the dataset first using                                    
                               CompetitionSpecification.cache() for more performant data access.                   

In [2]:
poses = train.as_array("Ligand Pose")
clean_poses = [Chem.RemoveHs(m) for m in poses]
smiles = [Chem.MolToSmiles(m) for m in clean_poses]
mols = [Chem.MolFromSmiles(smi) for smi in smiles]
test_mols = []
targets = []
for t in test:
    test_mols.append(Chem.MolFromSmiles(t["CXSMILES"]))
    targets.append(t["Protein Label"])

# find MCS

In [3]:
from rdkit.Chem import rdRascalMCES
import tqdm
MCSs = []
opts = rdRascalMCES.RascalOptions()
opts.singleLargestFrag = True

mols_for_pred = test_mols

#find MCS
for i,m in enumerate(tqdm.tqdm(mols_for_pred)):
    curr_MCS = []
    curr_t = targets[i]
    opts.similarityThreshold = 0.9
    if curr_t == "MERS-CoV Mpro":
        mols_for_ref = mersmols
    else:
        mols_for_ref = sarsmols
    while len(curr_MCS)==0:
        for j,m2 in enumerate(mols_for_ref):
            mcr =  rdRascalMCES.FindMCES(m,m2,opts)
            if len(mcr)>0:
                curr_MCS.append((len(mcr[0].atomMatches()),mcr[0].smartsString,j))
        if len(curr_MCS) > 0: #to prevent artefacts
            if len(sorted(curr_MCS)[-1][1]) < 10 and opts.similarityThreshold>0.2:
                curr_MCS = []
        opts.similarityThreshold += -0.1
    MCSs.append(sorted(curr_MCS)[-1])
    print(i,sorted(curr_MCS)[-1],Chem.MolToSmiles(m),opts.similarityThreshold)

  3%|█                                          | 5/195 [00:00<00:03, 47.57it/s]

0 (16, '[#6]-CC(=O)-N(-c1:c:n:c:c2:c:c:c:c:c:1:2)-C', 0) O=C1c2ccc(Cl)cc2[C@@]2(CN1Cc1nnco1)C(=O)N(c1cncc3ccccc13)C[C@@H]2CNc1ncncn1 0.5000000000000001
1 (10, 'c1:c:n:c:c2:c:c:c:c:c:1:2', 0) COc1ccc(N(Cc2cccc(Cl)c2)C(=O)Cc2cncc3ccccc23)cc1 0.6000000000000001
2 (32, 'CNC(=O)-CN1CC2(-C(=O)-N(-c3:c:n:c:c4:c:c:c:c:c:3:4)-CC2)-c2:c:c(-Cl):c:c:c:2-C1=O', 453) CNC(=O)CN1C[C@@]2(C(=O)N(c3cncc4ccccc34)C[C@@H]2CNc2ccncn2)c2cc(Cl)ccc2C1=O 0.7000000000000001
3 (16, 'C(-C(=O)-N(-c1:c:n:c:c2:c:c:c:c:c:1:2)-C)-[#6]', 0) CNC(=O)CN1C[C@@]2(C(=O)N(c3cncc4ccccc34)C[C@@H]2[NH3+])c2cc(Cl)ccc2C1=O 0.6000000000000001
4 (16, '[#6]-CC(=O)-N(-c1:c:n:c:c2:c:c:c:c:c:1:2)-C', 0) O=C1c2ccc(Cl)cc2[C@@]2(CN1Cc1ccnnc1)C(=O)N(c1cncc3ccccc13)C[C@@H]2CNc1ccncn1 0.5000000000000001
5 (32, 'CNC(=O)-CN1CC2(-C(=O)-N(-c3:c:n:c:c4:c:c:c:c:c:3:4)-CC2)-c2:c:c(-Cl):c:c:c:2-C1=O', 453) CNC(=O)CN1C[C@@]2(C(=O)N(c3cncc4ccccc34)C[C@@H]2CNc2cnn(C)c2)c2cc(Cl)ccc2C1=O 0.7000000000000001
6 (28, '[#6]-CN1CC2(-C(=O)-N(-c3:c:n:c:c4:c:c:c:c:c

  5%|██▏                                       | 10/195 [00:00<00:14, 13.19it/s]

8 (27, 'C1CN(-c2:c:n:c:c3:c:c:c:c:c:2:3)-C(=O)-C12CN(-C-[#6])-Cc1:c:c:c:c:c-2:1', 655) C[C@H]1CN(c2cncc3ccccc23)C(=O)[C@@]12CN(Cc1nccn1C)C(=O)c1ccc(F)cc12 0.7000000000000001
9 (10, 'c1:c:n:c:c2:c:c:c:c:c:1:2', 0) C[C@H]1c2ccsc2CCN1C(=O)Cc1cncc2ccccc12 0.5000000000000001
10 (16, 'CN(-c1:c:n:c:c2:c:c:c:c:c:1:2)-C(=O)-C-[#6]', 0) C[C@H]1CN(c2cncc3ccccc23)C(=O)[C@@]12CN(CC#N)C(=O)c1ccc(F)cc12 0.6000000000000001
11 (12, 'c1:c(-N(-Cc)-C(=O)-C):c:c:c:c:1', 4) CCC(=O)[N@H+]1CCCc2c(N(Cc3cccc(Cl)c3)C(=O)Cc3cncc4ccccc34)cccc21 0.5000000000000001


  8%|███▍                                      | 16/195 [00:01<00:11, 15.32it/s]

...

184 (29, 'C1CN(-c2:c:n:c:c3:c:c:c:c:c:2:3)-C(=O)-C12CN(-C-[#6])-C(=O)-c1:c:c:c(-Cl):c:c-2:1', 640) COC[C@H]1CN(c2cncc3ccccc23)C(=O)[C@@]12CN(Cc1cn[nH]c1)C(=O)c1ccc(Cl)cc12 0.7000000000000001
185 (10, 'c1:c:n:c:c2:c:c:c:c:c:1:2', 0) O=C(Cc1cncc2ccccc12)N1C=CC=CC=C[C@H]1c1ccccc1 0.6000000000000001
186 (16, 'C(-C(=O)-N(-c1:c:n:c:c2:c:c:c:c:c:1:2)-C)-[#6]', 0) C[NH2+]CCN1C[C@@]2(C(=O)N(c3cncc4ccccc34)C[C@@H]2C)c2cc(F)ccc2C1=O 0.6000000000000001
187 (30, 'Cn1:n:c:c:c:1-CN1CC(-C(=O)-Nc2:c:n:c:c3:c:c:c:c:c:2:3)-c2:c:c:c:c:c:2-C1', 643) CCn1nccc1CN1C[C@@]2(C(=O)N(c3cncc4ccccc34)C[C@@H]2C)c2cc(F)ccc2C1=O 0.7000000000000001
188 (12, 'c1:c(-N(-Cc)-C(=O)-C):c:c:c:c:1', 4) CCC(=O)[N@@H+]1CCCc2c(N(Cc3cccc(Cl)c3)C(=O)Cc3cncc4ccccc34)cccc21 0.5000000000000001
189 (16, 'C(-C(=O)-N(-c1:c:n:c:c2:c:c:c:c:c:1:2)-C)-[#6]', 0) CNC(=O)CN1C[C@@]2(C(=O)N(c3cncc4ccccc34)C[C@@H]2CS(=O)(=O)N(C)C)c2cc(F)ccc2C1=O 0.5000000000000001


 98%|████████████████████████████████████████▎| 192/195 [00:13<00:00, 14.70it/s]

190 (13, 'CC(=O)-NCc1:c:c(-Cl):c:c(-[#6]):c:1', 242) CNS(=O)(=O)OCC(=O)N[C@@H](C)c1cc(Cl)cc(-c2ccc(S(N)(=O)=O)cc2)c1 0.5000000000000001
191 (32, 'CNC(=O)-CN1CC2(-C(=O)-N(-c3:c:n:c:c4:c:c:c:c:c:3:4)-CC2)-c2:c:c(-Cl):c:c:c:2-C1=O', 453) CNC(=O)CN1C[C@@]2(C(=O)N(c3cncc4ccccc34)C[C@@H]2CNc2ccncn2)c2cc(Cl)ccc2C1=O 0.7000000000000001
192 (10, 'c1:c:n:c:c2:c:c:c:c:c:1:2', 0) O=C(Cc1cncc2ccccc12)NCc1cccc(Cl)c1 0.6000000000000001


100%|█████████████████████████████████████████| 195/195 [00:13<00:00, 14.48it/s]

193 (32, 'CNC(=O)-CN1CC2(-C(=O)-N(-c3:c:n:c:c4:c:c:c:c:c:3:4)-CC2)-c2:c:c(-Cl):c:c:c:2-C1=O', 640) CNC(=O)CN1C[C@@]2(C(=O)N(c3cncc4ccccc34)C[C@@H]2COC(C)C)c2cc(Cl)ccc2C1=O 0.7000000000000001
194 (20, 'CCC1(-C)-NC(=O)-N(-c2:c:n:c:c3:c:c:c:c:c:2:3)-C1=O', 379) C[C@@H]1COC[C@@]2(C1)NC(=O)N(c1cncc3ccccc13)C2=O 0.7000000000000001


In [4]:
from rdkit.Chem import rdDistGeom,rdMolAlign,rdShapeAlign
import copy

def shape_compare(ref,prb,prbId=0):
    """
    crude volume compare to pick out most overlapping pose without translation or rotation
    round coords to 3d grid and count common occupied points in grid
    """
    ref_atoms = []
    for a in ref.GetAtoms():
        coords = list(ref.GetConformer(0).GetAtomPosition(a.GetIdx()))
        ref_atoms.append(tuple([round(c,0)//1 for c in coords]))
    ref_grid = set(ref_atoms)
    prb_atoms = []
    for a in prb.GetAtoms():
        coords = list(prb.GetConformer(prbId).GetAtomPosition(a.GetIdx()))
        prb_atoms.append(tuple([round(c,0)//1 for c in coords]))
    prb_grid = set(prb_atoms)
    return len(list(prb_grid.intersection(ref_grid)))/len(list(ref_grid))

ps = rdDistGeom.ETKDGv3()
ps.randomSeed = 0xc0ffee
ps.numThreads = 12
ps.enforceChirality = False

n_conformers = 50

def get_pose(mol,ref,smarts):
    p = Chem.MolFromSmarts(smarts)
    tm = mol.GetSubstructMatch(p)
    rm = ref.GetSubstructMatch(p)
    r = -1
    lrm = 0
    while r==-1:
        cmap = {}
        for i,t in enumerate(tm[lrm:]):
            cmap[t] = ref.GetConformer(0).GetAtomPosition(rm[i])
        newm = Chem.AddHs(mol)
        ps.coordMap = cmap
        r = rdDistGeom.EmbedMultipleConfs(newm,n_conformers,ps)
        lrm += 1
    for i in range(n_conformers):
        rdMolAlign.AlignMol(newm,ref,atomMap = list(zip(tm,rm)),prbCid=i)
    newmc = copy.deepcopy(newm)
    bestidx = 0
    bestoverlap = 0
    overlaps = []
    for i in range(n_conformers):
        overlap = shape_compare(ref, newmc, prbId=i)
        if overlap > bestoverlap:
            bestoverlap = overlap
            bestidx = i
        overlaps.append(overlap)
    newm_best = Chem.Mol(newm,confId=bestidx)
    return newm_best

final_poses = []
for i,MCS in enumerate(tqdm.tqdm(MCSs)):
    if targets[i] == "SARS-CoV-2 Mpro":
        final_poses.append(get_pose(mols_for_pred[i],sarsmols[MCS[2]],MCS[1]))
    else:
        final_poses.append(get_pose(mols_for_pred[i],mersmols[MCS[2]],MCS[1]))
     

100%|█████████████████████████████████████████| 195/195 [04:55<00:00,  1.51s/it]


In [8]:
import base64
from rdkit import Chem


def serialize_rdkit_mol(mol: Chem.Mol): 
    props = Chem.PropertyPickleOptions.AllProps
    mol_bytes = mol.ToBinary(props)
    return base64.b64encode(mol_bytes).decode('ascii')  


y_pred_serialized = [serialize_rdkit_mol(mol) for mol in final_poses]


In [9]:
wri = Chem.SDWriter("final_preds.sdf")
for m in final_poses:
    wri.write(m)
wri.close()

In [10]:
competition.submit_predictions(
    predictions=y_pred_serialized,
    prediction_name="MCES_alignment3",
    prediction_owner="wim0",
    report_url="https://molecular.beauty/blog/2025/03/14/polaris.html", 
    # The below metadata is optional, but recommended.
    github_url="https://github.com/dehaenw/polaris-baseline",
    description="Source code and report will be made public once challenge is finished",
    user_attributes={"Framework": "RDKit", "Method": "MCES constained embedding. pose selection by volume overlap"}
)



Output()

[18:12:31]  Success: Submitting competition predictions                                               ]8;id=806688;file:///home/wim/anaconda3/envs/my-rdkit-env/lib/python3.12/site-packages/polaris/utils/context.py\context.py]8;;\:]8;id=474784;file:///home/wim/anaconda3/envs/my-rdkit-env/lib/python3.12/site-packages/polaris/utils/context.py#53\53]8;;\